In [51]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch
from api.core.mongo import db
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_nomic import NomicEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [44]:
llm = ChatOpenAI()
vector_store = MongoDBAtlasVectorSearch(
    collection=db["embeddings"], 
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local", dimensionality=768),
    index_name="vector_index", 
    relevance_score_fn="euclidean",
    text_key="text",
)

In [45]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": vector_store.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

C:\Users\holmt\PycharmProjects\sunhacks-2024\venv\lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [50]:
#: find documents based on a query
vector_store.similarity_search("What are your most expensive phone?", k=3)

[Document(metadata={'_id': '66f98aca081255c9e70a616b'}, page_content=''),
 Document(metadata={'_id': '66f98744081255c9e70a598d'}, page_content=''),
 Document(metadata={'_id': '66f987f3081255c9e70a5b29'}, page_content='')]

In [53]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(vector_store.as_retriever(), combine_docs_chain)
rag_chain.invoke({"input": "What phone should I get? I want an iPhone."})

C:\Users\holmt\PycharmProjects\sunhacks-2024\venv\lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


{'input': 'What phone should I get? I want an iPhone.',
 'context': [Document(metadata={'_id': '66f98680081255c9e70a57da'}, page_content=''),
  Document(metadata={'_id': '66f98a10081255c9e70a5fd1'}, page_content=''),
  Document(metadata={'_id': '66f98a3d081255c9e70a602b'}, page_content=''),
  Document(metadata={'_id': '66f98a10081255c9e70a5fce'}, page_content='')],
 'answer': 'Based on your preference for an iPhone, you could consider the latest models such as the iPhone 13, iPhone 13 Pro, or iPhone 13 Mini. Each of these models offers different features and price points, so you can choose based on your specific needs and budget.'}